In [77]:
import pandas as pd
import ast

df = pd.read_csv('data/train.csv')

In [78]:
# problems 열 문자열을 딕셔너리로 변환
df['problems'] = df['problems'].apply(ast.literal_eval)

# answer 값 추출 및 개수 세기
answer_counts = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts)

problems
1    948
2    450
3    319
4    283
5     31
Name: count, dtype: int64


In [79]:
# choices 길이 계산 및 분포 확인
df['choices_length'] = df['problems'].apply(lambda x: len(x['choices']))

# 선택지 길이별 분포 확인
choices_length_distribution = df['choices_length'].value_counts()

# 결과 출력
print(choices_length_distribution)


choices_length
5    1239
4     792
Name: count, dtype: int64


In [80]:
# 948 - 541 = 407
# 450 - 44 = 406
# 319 + 43 + 44 = 406
# 283 + 123 = 406
# 31 + 375 = 406

In [81]:
df_1 = df[df['problems'].apply(lambda x: x['answer'] == 1)]

# 인덱스를 정렬하여 항상 같은 순서를 보장
df_1 = df_1.sort_index()

# 선택지 길이가 5인 행만 필터링
df_choices_length_5 = df_1[df_1['choices_length'] == 5]

# 375개 행을 랜덤하게 선택 (고정된 시드값을 사용하여 항상 동일한 샘플링)
sampled_rows_1to5 = df_choices_length_5.sample(n=375, random_state=42)

# choices 리스트의 첫 번째와 다섯 번째 선택지 위치 변경
def swap_choices(problems):
    choices = problems['choices']
    if len(choices) >= 5:  # 다섯 번째 선택지가 존재할 경우에만 처리
        choices[0], choices[4] = choices[4], choices[0]
    problems['choices'] = choices
    return problems

sampled_rows_1to5['problems'] = sampled_rows_1to5['problems'].apply(swap_choices)

# answer 값을 5로 변경하는 함수
def set_answer_to_5(problems):
    problems['answer'] = 5  # answer 값을 5로 설정
    return problems

# sampled_rows에서 answer 값을 5로 변경
sampled_rows_1to5['problems'] = sampled_rows_1to5['problems'].apply(set_answer_to_5)

sampled_rows_1to5 = sampled_rows_1to5.drop(columns=['choices_length'])

print(sampled_rows_1to5.head())

                           id  \
1384  generation-for-nlp-2117   
1217  generation-for-nlp-1913   
2002  generation-for-nlp-2865   
1110  generation-for-nlp-1780   
1336  generation-for-nlp-2060   

                                              paragraph  \
1384  사회적 거리두기가 일상화되면서 사무실, 공장, 음식점 등 다중이용시설에서 ‘아크릴 ...   
1217  지난해 3차 양적완화의 필요성을 강하게 주장했던 미국 중앙은행(Fed)의 대표적 비...   
2002  1995년 외식체인점 후터스의 미국 플로리다 잭슨빌 매장에 17세의 갈색머리 소녀가...   
1110  허니비즈(대표 윤문진)는 서울시 ‘서울사랑상품권’에 이어, 경기도 시흥시와 지역화폐...   
1336  핀란드 관광청이 3일 온라인 브리핑을 통해 지속가능한 핀란드 여행(Sustainab...   

                                               problems  question_plus  
1384  {'question': '애드피아몰이 출시한 아크릴 가림막의 주된 용도는 무엇인가?...            NaN  
1217  {'question': '윌리엄스 총재가 3차 양적완화의 속도 조절 가능성을 제기한...            NaN  
2002  {'question': '카트리나 콜이 후터스에서 일하기 시작한 이유는 무엇인가?'...            NaN  
1110  {'question': '허니비즈가 시흥시와 체결한 협약의 주요 목적은 무엇인가?'...            NaN  
1336  {'question': '핀란드 관광청이 공식 론칭한 프로그램의 이름은 무엇인가?'...            NaN  


In [82]:
df_1to5 = sampled_rows_1to5[sampled_rows_1to5['problems'].apply(lambda x: x['answer'] == 5)]
df_1to5

,id,paragraph,problems,question_plus
1384,generation-for-nlp-2117,"사회적 거리두기가 일상화되면서 사무실, 공장, 음식점 등 다중이용시설에서 ‘아크릴 ...",{'question': '애드피아몰이 출시한 아크릴 가림막의 주된 용도는 무엇인가?...,NaN
1217,generation-for-nlp-1913,지난해 3차 양적완화의 필요성을 강하게 주장했던 미국 중앙은행(Fed)의 대표적 비...,{'question': '윌리엄스 총재가 3차 양적완화의 속도 조절 가능성을 제기한...,NaN
2002,generation-for-nlp-2865,1995년 외식체인점 후터스의 미국 플로리다 잭슨빌 매장에 17세의 갈색머리 소녀가...,{'question': '카트리나 콜이 후터스에서 일하기 시작한 이유는 무엇인가?'...,NaN
1110,generation-for-nlp-1780,"허니비즈(대표 윤문진)는 서울시 ‘서울사랑상품권’에 이어, 경기도 시흥시와 지역화폐...",{'question': '허니비즈가 시흥시와 체결한 협약의 주요 목적은 무엇인가?'...,NaN
1336,generation-for-nlp-2060,핀란드 관광청이 3일 온라인 브리핑을 통해 지속가능한 핀란드 여행(Sustainab...,{'question': '핀란드 관광청이 공식 론칭한 프로그램의 이름은 무엇인가?'...,NaN
...,...,...,...,...
2029,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,{'question': '방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?'...,NaN
1824,generation-for-nlp-2654,"지역별로는 수도권이 1월보다 3419가구 줄어든 2만9278가구를 기록, 4개월 연...","{'question': '미분양 주택이 줄어든 주된 이유는 무엇인가?', 'choi...",NaN
991,generation-for-nlp-1635,"한우산업의 안정적 발전을 위해 설립된 한우자조금관리위원회(위원장 민경천, 이하 한우...",{'question': '2020 한우영상공모전에서 1등을 수상한 작품의 제목은 무...,NaN
901,generation-for-nlp-1519,"㈜엔씨소프트(대표 김택진, 이하 엔씨(NC))는 고용노동부에서 주관하는 ‘2020 ...",{'question': '㈜엔씨소프트가 2020 대한민국 일자리 으뜸기업으로 선정된...,NaN


In [83]:
df = df.drop(columns=['choices_length'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


In [84]:
# sampled_rows의 인덱스를 가져와서 df에서 해당 인덱스를 제거
df = df.drop(sampled_rows_1to5.index)

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_5 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_5)

problems
1    573
2    450
3    319
4    283
5     31
Name: count, dtype: int64


In [85]:
df_1 = df[df['problems'].apply(lambda x: x['answer'] == 1)]

# 인덱스를 정렬하여 항상 같은 순서를 보장
df_1 = df_1.sort_index()

# 375개 행을 랜덤하게 선택 (고정된 시드값을 사용하여 항상 동일한 샘플링)
sampled_rows_1to4 = df_1.sample(n=123, random_state=42)

# choices 리스트의 첫 번째와 다섯 번째 선택지 위치 변경
def swap_choices(problems):
    choices = problems['choices']
    choices[0], choices[3] = choices[3], choices[0]
    problems['choices'] = choices
    return problems

sampled_rows_1to4['problems'] = sampled_rows_1to4['problems'].apply(swap_choices)

# answer 값을 5로 변경하는 함수
def set_answer_to_4(problems):
    problems['answer'] = 4  # answer 값을 5로 설정
    return problems

# sampled_rows에서 answer 값을 5로 변경
sampled_rows_1to4['problems'] = sampled_rows_1to4['problems'].apply(set_answer_to_4)

print(sampled_rows_1to4.head())

                           id  \
1839  generation-for-nlp-2671   
312    generation-for-nlp-806   
601   generation-for-nlp-1153   
1535  generation-for-nlp-2303   
1943  generation-for-nlp-2793   

                                              paragraph  \
1839  푸마(PUMA)가 지난 12일(한국시간), 전 세계 축구 슈퍼스타 중 한 명인 ‘네...   
312   낸시는 1시간 동안 시험 공부를 하거나, 6달러의 임금을 받고 1시간 동안 잔디를 ...   
601   이 질문은 다음 정보를 참조한다.\n그는 나를 붙잡고 어둠의 집으로 끌고갔다…\n한...   
1535  “노조 파업으로 자동차 생산라인이 멈춰서고, 하청업체는 일감 부족으로 고통을 받는 ...   
1943  극심한 청년 취업난 속 취업준비생들은 상반기 공채시즌에 집중하고 있다. 또한 회계사...   

                                               problems  question_plus  
1839  {'question': '네이마르가 푸마와 체결한 파트너십의 주요 내용은 무엇인가?...            NaN  
312   {'question': '낸시가 시험 공부를 하는 것을 선택했다는 것을 알고 있다면...            NaN  
601   {'question': '다음 중 위 두 개의 글이 가장 잘 뒷받침하는 결론은 무엇...            NaN  
1535  {'question': '현대차 노조의 최근 파업 결정에 대한 반발을 나타낸 인물은...            NaN  
1943  {'question': 'KG에듀원 미래경영아카데미가 실시하는 모의고사의 주요 목적...            NaN  


In [86]:
df_1to4 = sampled_rows_1to4[sampled_rows_1to4['problems'].apply(lambda x: x['answer'] == 4)]
df_1to4

,id,paragraph,problems,question_plus
1839,generation-for-nlp-2671,"푸마(PUMA)가 지난 12일(한국시간), 전 세계 축구 슈퍼스타 중 한 명인 ‘네...",{'question': '네이마르가 푸마와 체결한 파트너십의 주요 내용은 무엇인가?...,NaN
312,generation-for-nlp-806,"낸시는 1시간 동안 시험 공부를 하거나, 6달러의 임금을 받고 1시간 동안 잔디를 ...",{'question': '낸시가 시험 공부를 하는 것을 선택했다는 것을 알고 있다면...,NaN
601,generation-for-nlp-1153,이 질문은 다음 정보를 참조한다.\n그는 나를 붙잡고 어둠의 집으로 끌고갔다…\n한...,{'question': '다음 중 위 두 개의 글이 가장 잘 뒷받침하는 결론은 무엇...,NaN
1535,generation-for-nlp-2303,"“노조 파업으로 자동차 생산라인이 멈춰서고, 하청업체는 일감 부족으로 고통을 받는 ...",{'question': '현대차 노조의 최근 파업 결정에 대한 반발을 나타낸 인물은...,NaN
1943,generation-for-nlp-2793,극심한 청년 취업난 속 취업준비생들은 상반기 공채시즌에 집중하고 있다. 또한 회계사...,{'question': 'KG에듀원 미래경영아카데미가 실시하는 모의고사의 주요 목적...,NaN
...,...,...,...,...
271,generation-for-nlp-755,정부가 방금 개인 소득세를 인하했습니다.,{'question': '다음 중 이 정책이 경제에 미치는 영향에 대해 가장 잘 설...,NaN
86,generation-for-nlp-529,먼저 우주가 구형이라는 점에 주목해야 한다. 부분적으로는 연결점이 필요 없는 완전한...,{'question': '위 글을 보았을 때 구체 형태의 우주에 관한 코페르니쿠스의...,NaN
1798,generation-for-nlp-2619,서울중앙지검 금융조세조사2부(부장검사 김범기)는 30일 가전업체 모뉴엘로부터 금품을...,{'question': '조계륭 전 한국무역보험공사 사장이 구속영장이 청구된 이유는...,NaN
620,generation-for-nlp-1175,프랑스 혁명의 위대한 순수성이야말로 우리의 약점과 강점 모두를 끌어내는 것입니다. ...,"{'question': '위 글의 화자가 가장 염려하고 있는 위협은 무엇입니까?',...",NaN


In [87]:
# sampled_rows의 인덱스를 가져와서 df에서 해당 인덱스를 제거
df = df.drop(sampled_rows_1to4.index)

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_4 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_4)

problems
2    450
1    450
3    319
4    283
5     31
Name: count, dtype: int64


In [88]:
df_1 = df[df['problems'].apply(lambda x: x['answer'] == 1)]

# 인덱스를 정렬하여 항상 같은 순서를 보장
df_1 = df_1.sort_index()

# 375개 행을 랜덤하게 선택 (고정된 시드값을 사용하여 항상 동일한 샘플링)
sampled_rows_1to3 = df_1.sample(n=43, random_state=42)

# choices 리스트의 첫 번째와 다섯 번째 선택지 위치 변경
def swap_choices(problems):
    choices = problems['choices']
    choices[0], choices[2] = choices[2], choices[0]
    problems['choices'] = choices
    return problems

sampled_rows_1to3['problems'] = sampled_rows_1to3['problems'].apply(swap_choices)

# answer 값을 3으로 변경하는 함수
def set_answer_to_3(problems):
    problems['answer'] = 3  # answer 값을 3로 설정
    return problems

# sampled_rows에서 answer 값을 3로 변경
sampled_rows_1to3['problems'] = sampled_rows_1to3['problems'].apply(set_answer_to_3)

print(sampled_rows_1to3.head())

                           id  \
1847  generation-for-nlp-2679   
1996  generation-for-nlp-2857   
691   generation-for-nlp-1257   
172    generation-for-nlp-631   
1877  generation-for-nlp-2717   

                                              paragraph  \
1847  전국 최대 검찰청인 서울중앙지검은 13일 차장·부장검사가 중요 사건을 직접 수사하는...   
1996  미국에 이어 스페인에서도 에볼라 바이러스에 감염된 환자가 확인됐다.월스트리트저널(W...   
691   린 쩌쉬, 중국 무역 대표, 빅토리아 여왕에게 보낸 편지, 1839. 그렇다면 어떤...   
172   이 지문에 나타난 정보는 무엇입니까? 다음 편지를 읽어보십시오. 사건은 . . . ...   
1877  정부가 손자병법까지 인용하며 외환시장의 불안심리를 선제적으로 차단하고 투기세력은 강...   

                                               problems  question_plus  
1847  {'question': '서울중앙지검의 수사 개선안의 주된 목적은 무엇인가?', '...            NaN  
1996  {'question': '스페인에서 에볼라 바이러스에 감염된 간호사가 치료한 환자의...            NaN  
691   {'question': '다음 중 린 쩌쉬의 편지가 전달된 후 뒤따른 결과는?', ...            NaN  
172   {'question': '다음 중 17세기 말 실현된 유럽의 아메리카 정복의 결과가...            NaN  
1877  {'question': '추경호 기획재정부 제1차관이 인용한 손자병법의 문장은 어떤...            NaN  


In [89]:
df_1to3 = sampled_rows_1to3[sampled_rows_1to3['problems'].apply(lambda x: x['answer'] == 3)]
len(df_1to3)

43

In [90]:
# sampled_rows의 인덱스를 가져와서 df에서 해당 인덱스를 제거
df = df.drop(sampled_rows_1to3.index)

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_3 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_3)

problems
2    450
1    407
3    319
4    283
5     31
Name: count, dtype: int64


In [91]:
df_2 = df[df['problems'].apply(lambda x: x['answer'] == 2)]

# 인덱스를 정렬하여 항상 같은 순서를 보장
df_2 = df_2.sort_index()

# 375개 행을 랜덤하게 선택 (고정된 시드값을 사용하여 항상 동일한 샘플링)
sampled_rows_2to3 = df_2.sample(n=44, random_state=42)

# choices 리스트의 첫 번째와 다섯 번째 선택지 위치 변경
def swap_choices(problems):
    choices = problems['choices']
    choices[1], choices[2] = choices[2], choices[1]
    problems['choices'] = choices
    return problems

sampled_rows_2to3['problems'] = sampled_rows_2to3['problems'].apply(swap_choices)

# answer 값을 3으로 변경하는 함수
def set_answer_to_3(problems):
    problems['answer'] = 3  # answer 값을 3로 설정
    return problems

# sampled_rows에서 answer 값을 3로 변경
sampled_rows_2to3['problems'] = sampled_rows_2to3['problems'].apply(set_answer_to_3)

print(sampled_rows_2to3.head())

                           id  \
1813  generation-for-nlp-2641   
1994  generation-for-nlp-2855   
522   generation-for-nlp-1056   
129    generation-for-nlp-577   
1849  generation-for-nlp-2681   

                                              paragraph  \
1813  한국이 쌀 시장 개방을 미루려면 이해 당사국들에 보상을 해야 한다고 호베르토 아제베...   
1994  공급 과잉 논란 속에 배후 임대수요 확보 여부가 ‘수익형 부동산’인 오피스텔 선택 ...   
522   종교적 직업과 예배에서 양심의 자유가 제한된다면 시민의 자유를 최대한 누리더라도 진...   
129   "중국 황제 폐하께서는 영국 백성이 가족 및 시설과 함께 상업을 수행할 목적으로 박...   
1849  애플이 사상 처음으로 시가총액 7000억달러(약 774조원) 고지를 찍으면서 미국 ...   

                                               problems  question_plus  
1813  {'question': '호베르토 아제베도 WTO 사무총장이 언급한 한국의 쌀 관세...            NaN  
1994  {'question': '오피스텔 투자에서 안정적인 임대수요를 확보하기 위해 가장 ...            NaN  
522   {'question': '다음 중 펜실베이니아 식민지를 가장 잘 설명한 것은 무엇입...            NaN  
129   {'question': '이 조약에서 제안된 양해가 다음 중 영국의 무엇을 직접적으...            NaN  
1849  {'question': '애플의 시가총액이 7000억 달러에 도달한 후, 주가 상승...            NaN  


In [92]:
df_2to3 = sampled_rows_2to3[sampled_rows_2to3['problems'].apply(lambda x: x['answer'] == 3)]
len(df_2to3)

44

In [93]:
# sampled_rows의 인덱스를 가져와서 df에서 해당 인덱스를 제거
df = df.drop(sampled_rows_2to3.index)

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_4 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_4)

problems
1    407
2    406
3    319
4    283
5     31
Name: count, dtype: int64


In [94]:
# 여러 데이터프레임 합치기
merged_sampled_rows = pd.concat([sampled_rows_1to5, sampled_rows_1to4, sampled_rows_1to3, sampled_rows_2to3])

# df와 합치기
df = pd.concat([df, merged_sampled_rows])

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_5 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_5)

problems
1    407
2    406
4    406
3    406
5    406
Name: count, dtype: int64


In [95]:
# id 열 기준으로 정렬
df = df.sort_values(by='id')

# 결과 확인
print(df)  # 정렬된 df 확인


                          id  \
483  generation-for-nlp-1000   
484  generation-for-nlp-1001   
485  generation-for-nlp-1003   
486  generation-for-nlp-1004   
487  generation-for-nlp-1005   
..                       ...   
478   generation-for-nlp-994   
479   generation-for-nlp-995   
480   generation-for-nlp-996   
481   generation-for-nlp-998   
482   generation-for-nlp-999   

                                             paragraph  \
483  "넓고 무한한 미래는 위대한 미국의 시대가 될 것이다. 많은 나라로 구성된 나라는 ...   
484  "기혼 여성의 재산을 보다 효과적으로 보호하기 위한 법률:\n"§1. 앞으로 결혼할...   
485  명백히 독재자 하나가 더는 침략에 맞서는 유엔 공동행동을 방해할 목적으로 동서 대결...   
486  상처를 입으면 불평할 권리가 있음은 모든 자유인이 주장하는 당연한 일이자, 특권이자...   
487  "우리는 25년 넘게 두 거대 정당이 권력을 얻고 빼앗으려 드는 동안 고통받는 자들...   
..                                                 ...   
478  그러는 동안 인디언들은 그들 주위를 조용히 찾아왔고 냉담한 모습을 보인 적도 있지만...   
479  “나는 한 때 억압의 도구였고 바보처럼 녹색이었다 독점 기업들은 담합해 나 같은 촌...   
480  "우리는 그것이 내전이었으며, 수년 동안 모든 식민지의 영향에서 해방을 추구해 온 ...   
481  자유인이 항상 가장 애써서 지켜온 권리 중 하나는 자신

In [ ]:
df.to_csv('data/train_balanced.csv', index=False, encoding='utf-8')